In [1]:
from get_sinks import get_sinks_from_content
import pandas as pd
import numpy as np
from extract_sink_features import sink_predict_class, SINK_TYPE_DICT

In [2]:
df = pd.read_csv('clean_datasets.csv')
print(df.columns)

opcode_list = df['opcode'].tolist()
code_list = df['code'].tolist()
filter_list = df['filter']
label_list = np.array(df['label'].tolist())
sink_type_list = []
for code in code_list:
    sink_type_list.append(get_sinks_from_content(code)[0])
sink_type_list = np.array(sink_type_list)

print(len(code_list))
print(len(sink_type_list))

Index(['file', 'code', 'vld', 'opcode', 'filter', 'label'], dtype='object')
10080
10080


In [7]:
# 替换SINK、ENCAPSED STRING

opcode_clean_list = []
for i in range(len(opcode_list)):
    opcode = str(opcode_list[i]).split(' ')
    for j in range(len(opcode)-1, -1, -1):
        if opcode[j] == 'ECHO':
            opcode[j] = SINK_TYPE_DICT[sink_predict_class(sink_type_list[i])]
            break

    opcode_clean_list.append(' '.join(opcode))
print(len(opcode_clean_list))

# 替换FUNCTION

for element in enumerate(zip(opcode_clean_list, filter_list)):
    index = element[0]    # Index of enumeration
    opcode_filter_tup = element[1]    # (opcode_sequence, filters)
    opcode = opcode_filter_tup[0].split(' ')    # to list
    filters = opcode_filter_tup[1].split(' ')    # to list
    count_init = 0
    count_do = 0
    # 无filter：
    if 'NO_FILTER' in filters:
        opcode_clean_list[index] = ' '.join(opcode)
        continue    # 跳转到下一份opcode序列
    # 有filter：
    for i in range(len(opcode)):    # 替换INIT_FCALL
        if opcode[i] == 'INIT_FCALL':
            opcode[i] = str(filters[count_init])
            count_init += 1
        if opcode[i] == 'DO_ICALL' and count_init != 0:    # 执行INIT_FCALL的地方必须在INIT_FCALL之后
            opcode[i] = 'DO_' + str(filters[count_do])
            count_do += 1
    opcode_clean_list[index] = ' '.join(opcode)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


10080


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
index = 19

print('>>> Filters: ')
print(filter_list[index] + '\n')
print('>>> Normal: ')
print(opcode_list[index] + '\n')
print('>>> Clean: ')
print(opcode_clean_list[index] + '\n')
# print(code_list[index] + '\n')
print(len(opcode_clean_list))

>>> Filters: 
EXEC FILTER_VAR FILTER_VAR

>>> Normal: 
ECHO ASSIGN INIT_FCALL SEND_VAR SEND_REF SEND_REF DO_ICALL FETCH_DIM_R ASSIGN INIT_FCALL SEND_VAR SEND_VAL DO_ICALL ASSIGN INIT_FCALL SEND_VAR SEND_VAL DO_ICALL JMPZ ASSIGN JMP ASSIGN ECHO ECHO RETURN

>>> Clean: 
ECHO ASSIGN EXEC SEND_VAR SEND_REF SEND_REF DO_EXEC FETCH_DIM_R ASSIGN FILTER_VAR SEND_VAR SEND_VAL DO_FILTER_VAR ASSIGN FILTER_VAR SEND_VAR SEND_VAL DO_FILTER_VAR JMPZ ASSIGN JMP ASSIGN ECHO ECHO_SINK_BODY_SECTION RETURN

10080


In [16]:
with open('./opcode_corpus/corpus.txt', encoding='utf-8', mode='w+') as f:
    for opcode in opcode_clean_list:
        f.write(opcode + '\n')
    f.close()

In [9]:
from gensim.models import Word2Vec
import nltk

corpus = []
for opcode in opcode_list:
#     corpus.append(nltk.word_tokenize(opcode))
    corpus.append(opcode.split(' '))
corpus = np.array(corpus)
w2v_model = Word2Vec(corpus, size=128, window=32, min_count=1, sg=0)

In [11]:
                               w2v_model.corpus_count

10080